# Google Maps Integration

This notebook shows how the [Google Maps API](https://developers.google.com/maps/) is integrated to plot a courier's path from a restaurant to a customer.

In [1]:
!umd --version

urban-meal-delivery, version 0.4.0


### Imports

In [2]:
import googlemaps as gm

from urban_meal_delivery import config, db

In [3]:
%load_ext lab_black

### Settings

Choose `"Bordeaux"`, `"Lyon"`, or `"Paris"`.

In [4]:
city_name = "Paris"

### Load the Data

In [5]:
city = db.session.query(db.City).filter_by(name=city_name).one()

In [6]:
city

<City(Paris)>

In [7]:
restaurants = (
    db.session.query(db.Restaurant)
    .join(db.Address)
    .filter(db.Address.city == city)
    .all()
)

In [8]:
len(restaurants)

1153

## Visualization

Let's choose a restaurant and then one of its orders.

In [9]:
restaurant = restaurants[0]

In [10]:
len(restaurant.orders)

3297

In [11]:
order = restaurant.orders[10]

Plot the courier's path from the restaurant to the customer's delivery address.

In [12]:
order.draw()

## Behind the Scenes

The code above integrates the [googlemaps](https://github.com/googlemaps/google-maps-services-python) PyPI package into the application running the UDP's routing optimization.

Let's first look at how we can use the [googlemaps](https://github.com/googlemaps/google-maps-services-python) PyPI package directly to make indiviual API calls.

Then, we'll see how this code is **abstracted** into the **application logic**.

### Direct Google Maps API Calls

With an API key, one can create a new `client` object that handles all communication with the [Google Maps API](https://developers.google.com/maps/).

In [13]:
client = gm.Client(config.GOOGLE_MAPS_API_KEY)

For example, we can **geocode** an address: This means that we obtain the **latitude-longitude** coordinates for a given *postal* address, and some other structured info.

In [14]:
client.geocode("Burgplatz 2, Vallendar")

[{'address_components': [{'long_name': '2',
    'short_name': '2',
    'types': ['street_number']},
   {'long_name': 'Burgplatz', 'short_name': 'Burgpl.', 'types': ['route']},
   {'long_name': 'Vallendar',
    'short_name': 'Vallendar',
    'types': ['locality', 'political']},
   {'long_name': 'Mayen-Koblenz',
    'short_name': 'Mayen-Koblenz',
    'types': ['administrative_area_level_3', 'political']},
   {'long_name': 'Rheinland-Pfalz',
    'short_name': 'RP',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Germany',
    'short_name': 'DE',
    'types': ['country', 'political']},
   {'long_name': '56179', 'short_name': '56179', 'types': ['postal_code']}],
  'formatted_address': 'Burgpl. 2, 56179 Vallendar, Germany',
  'geometry': {'bounds': {'northeast': {'lat': 50.40070650000001,
     'lng': 7.6142806},
    'southwest': {'lat': 50.4001466, 'lng': 7.6128609}},
   'location': {'lat': 50.40035899999999, 'lng': 7.613506600000001},
   'location_type': 'ROOFT

For routing applications, we are in particular interested in collecting all the *pair-wise* distances in a **distance matrix** for all involved **origins** and **destinations**.

In [15]:
restaurants[0].address

<Address(42 Rue De Charonne in Paris)>

In [16]:
restaurants[0].address.location.lat_lng

(48.85313, 2.37461)

In [17]:
origins = [
    restaurants[0].address.location.lat_lng,
    restaurants[1].address.location.lat_lng,
]

In [18]:
restaurants[0].orders[100].delivery_address

<Address(Boulevard De Menilmontant 83 in Paris)>

In [19]:
restaurants[0].orders[100].delivery_address.location.lat_lng

(48.864269, 2.385568)

In [20]:
destinations = [
    restaurants[0].orders[100].delivery_address.location.lat_lng,
    restaurants[1].orders[100].delivery_address.location.lat_lng,
]

The [Google Maps API](https://developers.google.com/maps/) provides a `.distance_matrix()` method for that.

In [21]:
client.distance_matrix(
    origins=origins,
    destinations=destinations,
    mode="bicycling",  # Choose an appropriate travelling mode
)

{'destination_addresses': ['83 Bd de Ménilmontant, 75011 Paris, France',
  '7 Rue des Abbesses, 75018 Paris, France'],
 'origin_addresses': ['19 Rue de Charonne, 75011 Paris, France',
  '64 Rue Saint-Lazare, 75009 Paris, France'],
 'rows': [{'elements': [{'distance': {'text': '2.4 km', 'value': 2389},
     'duration': {'text': '12 mins', 'value': 701},
     'status': 'OK'},
    {'distance': {'text': '5.8 km', 'value': 5798},
     'duration': {'text': '24 mins', 'value': 1454},
     'status': 'OK'}]},
  {'elements': [{'distance': {'text': '5.7 km', 'value': 5735},
     'duration': {'text': '22 mins', 'value': 1332},
     'status': 'OK'},
    {'distance': {'text': '1.2 km', 'value': 1211},
     'duration': {'text': '10 mins', 'value': 574},
     'status': 'OK'}]}],
 'status': 'OK'}

The `.directions()` method provides the **legs** (i.e., **waypoints**) of a **route** from *one* location to another.

In [22]:
client.directions(
    destination=restaurants[0].address.location.lat_lng,
    origin=restaurants[0].orders[100].delivery_address.location.lat_lng,
    mode="bicycling",  # Choose an appropriate travelling mode
)

[{'bounds': {'northeast': {'lat': 48.8643175, 'lng': 2.3887503},
   'southwest': {'lat': 48.8530147, 'lng': 2.3745434}},
  'copyrights': 'Map data ©2021',
  'legs': [{'distance': {'text': '2.0 km', 'value': 2008},
    'duration': {'text': '6 mins', 'value': 358},
    'end_address': '19 Rue de Charonne, 75011 Paris, France',
    'end_location': {'lat': 48.8530828, 'lng': 2.3745434},
    'start_address': '83 Bd de Ménilmontant, 75011 Paris, France',
    'start_location': {'lat': 48.8643175, 'lng': 2.3856677},
    'steps': [{'distance': {'text': '0.6 km', 'value': 554},
      'duration': {'text': '2 mins', 'value': 98},
      'end_location': {'lat': 48.85983239999999, 'lng': 2.3887503},
      'html_instructions': 'Head <b>southeast</b> on <b>Bd de Ménilmontant</b> toward <b>Rue de Tlemcen</b>/<wbr/><b>Rue Spinoza</b>',
      'polyline': {'points': '_yfiHm}pMt@{@`D{DJMBCh@m@HGDCLEjJsD`@MPGB?`@QjCaA'},
      'start_location': {'lat': 48.8643175, 'lng': 2.3856677},
      'travel_mode': 'BICY

### Abstraction as `Path` Objects

In the application's code base, the above API calls and the related data are modeled as `Path` objects connecting two `Address` objects (cf. [Path class](https://github.com/webartifex/urban-meal-delivery/blob/develop/src/urban_meal_delivery/db/addresses_addresses.py) in the code).

Let's look at two examples addresses, one from a `Restaurant` and one from a `Customer`.

In [23]:
restaurants[0].address

<Address(42 Rue De Charonne in Paris)>

In [24]:
restaurants[0].orders[100].delivery_address

<Address(Boulevard De Menilmontant 83 in Paris)>

The `Path.from_addresses()` constructor method takes any number of `Address` objects and creates all entries of a *symmetric* **distance matrix** as `Path` objects.

Here, we only get *one* `Path` object as there are only two `Address` objects.

In [25]:
paths = db.Path.from_addresses(
    restaurants[0].address, restaurants[0].orders[100].delivery_address
)

path = paths[0]

In [26]:
path

As we assume a *generic* and **symmetric** distance matrix, we call the two `Address` objects "first" and "second" and not "restaurant" and "customer". After all, a `Customer` may live in a house that has a `Restaurant` on the ground floor.

In [27]:
path.first_address

<Address(42 Rue De Charonne in Paris)>

In [28]:
path.second_address

<Address(Boulevard De Menilmontant 83 in Paris)>

Because we have `.latitude`-`.longitue` values for each `Address`, we can calculate the path's `.air_distance` even *without* talking to the Google Maps API.

In [29]:
path.air_distance

1475

The `Path.sync_with_google_maps()` method loads all the data needed from Google but makes sure that we do not make another API call if we already have the data.

In [30]:
path.sync_with_google_maps()

Google provides `.bicycle_distance` (in meters) and `.bicylce_duration` (in seconds) approximations for a courier's path from one location to another.

In [31]:
path.bicycle_distance

2389

In [32]:
path.bicycle_duration

702

In addition, the above `"legs"` values are stored as proper UTM coordinates for convenient plotting.

In [33]:
path.waypoints

[<Location: 31U 454118 5411312>,
 <Location: 31U 454126 5411304>,
 <Location: 31U 454063 5411179>,
 <Location: 31U 454238 5411110>,
 <Location: 31U 454501 5411786>,
 <Location: 31U 455184 5412061>,
 <Location: 31U 454979 5412537>,
 <Location: 31U 454966 5412524>,
 <Location: 31U 454944 5412554>]